In [50]:
import pandas as pd
from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler

folk = pd.read_csv('..\dataset generation\small_folk_features.csv')
folk['Class'] = 'folk'
folk = folk.iloc[2:, 1:]
#print(folk.head())
#print(folk.shape)

instrumental = pd.read_csv('..\dataset generation\small_instrumental_features.csv')
instrumental['Class'] = 'instrumental'
instrumental = instrumental.iloc[2:, 1:]
#print(instrumental.head())
#print(instrumental.shape)

hip_hop = pd.read_csv('..\dataset generation\small_hip_hop_features.csv')
hip_hop['Class'] = 'hip_hop'
hip_hop = hip_hop.iloc[2:, 1:]
#print(hip_hop.head())
#print(hip_hop.shape)

elec = pd.read_csv('..\dataset generation\small_elec_features.csv')
elec['Class'] = 'elec'
elec = elec.iloc[2:, 1:]
#print(elec.head())
#print(elec.shape)

rock = pd.read_csv('..\dataset generation\small_rock_features.csv')
rock['Class'] = 'rock'
rock = rock.iloc[2:, 1:]
#print(rock.head())
#print(rock.shape)

df = pd.concat([folk, instrumental, hip_hop, elec, rock])
df = df.sample(frac=1).reset_index(drop=True)
#print(df.head())
#print(df.shape)

col = df.columns.get_loc('mfcc')
mfcc = df.iloc[:, col:col+140]
clas = df['Class']
df = pd.concat([mfcc, clas], axis=1)

y_data = df['Class']
x_data = df.drop('Class', axis = 1)

class_data = pd.get_dummies(df['Class'], drop_first = False)

scaler = MinMaxScaler()
scaled = scaler.fit_transform(x_data)
x_df = DataFrame(scaled)

df = pd.concat([x_df, class_data], axis = 1)
df = df.dropna()

print(df.head())
print(df.shape)

          0         1         2         3         4         5         6  \
0  0.004381  0.006210  0.073670  0.102684  0.094826  0.130602  0.087988   
1  0.005598  0.011224  0.073573  0.084198  0.088598  0.200566  0.084707   
2  0.003647  0.006180  0.087575  0.109130  0.076379  0.112928  0.079403   
3  0.004930  0.019486  0.039732  0.092516  0.070580  0.106022  0.074359   
4  0.003548  0.014946  0.066461  0.139585  0.070036  0.198519  0.061189   

          7         8         9  ...       135       136       137       138  \
0  0.092990  0.078168  0.072337  ...  0.313985  0.345682  0.336967  0.323959   
1  0.101895  0.104864  0.099260  ...  0.301471  0.281096  0.280533  0.331763   
2  0.081040  0.089210  0.079731  ...  0.270845  0.260979  0.237212  0.301734   
3  0.081719  0.080296  0.068014  ...  0.404096  0.365145  0.331985  0.328452   
4  0.085848  0.089298  0.094656  ...  0.318527  0.361397  0.314246  0.387615   

        139  elec  folk  hip_hop  instrumental  rock  
0  0.327724  

In [51]:
from sklearn.model_selection import train_test_split

y_cols = ['folk', 'instrumental', 'hip_hop', 'rock', 'elec']
y_data = df[y_cols]
x_data = df.drop(y_data, axis = 1)

x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x_data, y_data, test_size = 0.1)

In [53]:
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(solver = 'sgd', random_state = 42, activation = 'logistic', learning_rate_init = 0.3, batch_size = 300, hidden_layer_sizes = (70,40), max_iter = 500)
mlp.fit(x_training_data, y_training_data)
predictions = mlp.predict(x_test_data)

print("Confusion Matrix:")
print(multilabel_confusion_matrix(y_test_data, predictions))
print("\nAccuracy:", accuracy_score(y_test_data, predictions))
print("\nMSE:", mean_squared_error(y_test_data, predictions))
print("\nClassification Report:")
print(classification_report(y_test_data, predictions))

Confusion Matrix:
[[[401  14]
  [ 24  61]]

 [[373  20]
  [ 60  47]]

 [[393  19]
  [ 32  56]]

 [[371  11]
  [ 37  81]]

 [[372  26]
  [ 42  60]]]

Accuracy: 0.6

MSE: 0.11399999999999999

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.72      0.76        85
           1       0.70      0.44      0.54       107
           2       0.75      0.64      0.69        88
           3       0.88      0.69      0.77       118
           4       0.70      0.59      0.64       102

   micro avg       0.77      0.61      0.68       500
   macro avg       0.77      0.61      0.68       500
weighted avg       0.77      0.61      0.68       500
 samples avg       0.60      0.61      0.61       500



c:\Users\micha\anaconda3\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
from sklearn.model_selection import GridSearchCV, KFold

learning_rate_init = [0.2, 0.3, 0.4]
max_iter = [300, 500, 700]
hidden_layer_sizes = [(70,40), (32,8), (100,60)]
solver = ['sgd']
random_state = [42]
activation = ['logistic']
batch_size = [300]


# Make a dictionary of the grid search parameters
param_grid = dict(learning_rate_init=learning_rate_init, max_iter=max_iter, hidden_layer_sizes=hidden_layer_sizes, solver=solver, random_state=random_state, activation=activation, batch_size=batch_size)

# Build and fit the GridSearchCV
grid = GridSearchCV(estimator=mlp, param_grid=param_grid,
                    cv=KFold(), verbose=10)

grid_results = grid.fit(x_data, y_data)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5; 1/27] START activation=logistic, batch_size=300, hidden_layer_sizes=(70, 40), learning_rate_init=0.2, max_iter=300, random_state=42, solver=sgd
[CV 1/5; 1/27] END activation=logistic, batch_size=300, hidden_layer_sizes=(70, 40), learning_rate_init=0.2, max_iter=300, random_state=42, solver=sgd;, score=0.524 total time=   2.0s
[CV 2/5; 1/27] START activation=logistic, batch_size=300, hidden_layer_sizes=(70, 40), learning_rate_init=0.2, max_iter=300, random_state=42, solver=sgd
[CV 2/5; 1/27] END activation=logistic, batch_size=300, hidden_layer_sizes=(70, 40), learning_rate_init=0.2, max_iter=300, random_state=42, solver=sgd;, score=0.582 total time=   2.2s
[CV 3/5; 1/27] START activation=logistic, batch_size=300, hidden_layer_sizes=(70, 40), learning_rate_init=0.2, max_iter=300, random_state=42, solver=sgd
[CV 3/5; 1/27] END activation=logistic, batch_size=300, hidden_layer_sizes=(70, 40), learning_rate_init=0.2, ma

c:\Users\micha\anaconda3\envs\env1\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 1/5; 10/27] END activation=logistic, batch_size=300, hidden_layer_sizes=(32, 8), learning_rate_init=0.2, max_iter=300, random_state=42, solver=sgd;, score=0.567 total time=   3.0s
[CV 2/5; 10/27] START activation=logistic, batch_size=300, hidden_layer_sizes=(32, 8), learning_rate_init=0.2, max_iter=300, random_state=42, solver=sgd
[CV 2/5; 10/27] END activation=logistic, batch_size=300, hidden_layer_sizes=(32, 8), learning_rate_init=0.2, max_iter=300, random_state=42, solver=sgd;, score=0.607 total time=   2.3s
[CV 3/5; 10/27] START activation=logistic, batch_size=300, hidden_layer_sizes=(32, 8), learning_rate_init=0.2, max_iter=300, random_state=42, solver=sgd
[CV 3/5; 10/27] END activation=logistic, batch_size=300, hidden_layer_sizes=(32, 8), learning_rate_init=0.2, max_iter=300, random_state=42, solver=sgd;, score=0.556 total time=   2.4s
[CV 4/5; 10/27] START activation=logistic, batch_size=300, hidden_layer_sizes=(32, 8), learning_rate_init=0.2, max_iter=300, random_state=42, 

c:\Users\micha\anaconda3\envs\env1\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 1/5; 25/27] END activation=logistic, batch_size=300, hidden_layer_sizes=(100, 60), learning_rate_init=0.4, max_iter=300, random_state=42, solver=sgd;, score=0.604 total time=   6.5s
[CV 2/5; 25/27] START activation=logistic, batch_size=300, hidden_layer_sizes=(100, 60), learning_rate_init=0.4, max_iter=300, random_state=42, solver=sgd
[CV 2/5; 25/27] END activation=logistic, batch_size=300, hidden_layer_sizes=(100, 60), learning_rate_init=0.4, max_iter=300, random_state=42, solver=sgd;, score=0.590 total time=   4.4s
[CV 3/5; 25/27] START activation=logistic, batch_size=300, hidden_layer_sizes=(100, 60), learning_rate_init=0.4, max_iter=300, random_state=42, solver=sgd
[CV 3/5; 25/27] END activation=logistic, batch_size=300, hidden_layer_sizes=(100, 60), learning_rate_init=0.4, max_iter=300, random_state=42, solver=sgd;, score=0.519 total time=   3.3s
[CV 4/5; 25/27] START activation=logistic, batch_size=300, hidden_layer_sizes=(100, 60), learning_rate_init=0.4, max_iter=300, rando

In [57]:
print("Optimal Paramaters:", grid_results.best_params_)
print("\nOptimal Hidden Layer Sizes:", grid_results.best_params_['hidden_layer_sizes'])
print("\nOptimal Learning Rate:", grid_results.best_params_['learning_rate_init'])
print("\nOptimal Number of Epochs:", grid_results.best_params_['max_iter'])
print("\nMax Accuracy:", grid_results.best_score_)

Optimal Paramaters: {'activation': 'logistic', 'batch_size': 300, 'hidden_layer_sizes': (32, 8), 'learning_rate_init': 0.2, 'max_iter': 500, 'random_state': 42, 'solver': 'sgd'}

Optimal Hidden Layer Sizes: (32, 8)

Optimal Learning Rate: 0.2

Optimal Number of Epochs: 500

Max Accuracy: 0.5826336336336336
